<a href="https://colab.research.google.com/github/Illumaria/iti_funds_nav/blob/master/iti_ruse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [0]:
#url = 'https://www.londonstockexchange.com/exchange/prices-and-markets/ETFs/company-news.html?fourWayKey=LU1483649312LUUSDEUET'
url = 'https://www.londonstockexchange.com/exchange/prices-and-markets/rns/company-news.html?tidm=RUSE&isin=LU1483649312&newsType='

In [0]:
iti_resp = requests.get(url)

In [0]:
bs = BeautifulSoup(iti_resp.text)

In [0]:
# Get the number of pages:
pages = bs.find('p', text = re.compile('Page ')).text
num_pages = int(pages.split()[3])

In [0]:
# Uncomment the following code if we need to take into account the NAV publication dates
# instead of their calculation dates:
#dates = bs.find_all('span', class_="hour")
#for date in dates:
#  print(date.text.split()[0]+" "+date.text.split()[1]+" "+date.text.split()[2])

In [0]:
# Find all the partial links on the first page:
scripts = bs.find_all('a', text = re.compile('Net Asset Value'))
#for script in scripts:
#        print(script.get('href').split("'")[1])

In [0]:
# Get the full links:
urls = []
urls.extend(['https://www.londonstockexchange.com' + scripts[i].get('href').split("'")[1] for i in range(len(scripts))])

In [0]:
# Do the same for other pages:
for i in range(2,num_pages+1):
  resp = requests.get('{}{}{}'.format(url, "&page=",i))
  bs = BeautifulSoup(resp.text)
  scripts = bs.find_all('a', text = re.compile('Net Asset Value'))
  urls.extend(['https://www.londonstockexchange.com' + scripts[i].get('href').split("'")[1] for i in range(len(scripts))])

In [10]:
for i in range(len(urls)):
  print('{}: {}'.format(i, urls[i]))

0: https://www.londonstockexchange.com/exchange/news/market-news/market-news-detail/RUSE/14429261.html
1: https://www.londonstockexchange.com/exchange/news/market-news/market-news-detail/RUSE/14427582.html
2: https://www.londonstockexchange.com/exchange/news/market-news/market-news-detail/RUSE/14425887.html
3: https://www.londonstockexchange.com/exchange/news/market-news/market-news-detail/RUSE/14424188.html
4: https://www.londonstockexchange.com/exchange/news/market-news/market-news-detail/RUSE/14422348.html
5: https://www.londonstockexchange.com/exchange/news/market-news/market-news-detail/RUSE/14420610.html
6: https://www.londonstockexchange.com/exchange/news/market-news/market-news-detail/RUSE/14418887.html
7: https://www.londonstockexchange.com/exchange/news/market-news/market-news-detail/RUSE/14417149.html
8: https://www.londonstockexchange.com/exchange/news/market-news/market-news-detail/RUSE/14415333.html
9: https://www.londonstockexchange.com/exchange/news/market-news/market-n

In [0]:
# THESE ARE OUTPUT ARRAYS:
dates = []
nav_per_share = []
num_shares = []
total_navs = []

# FILL THESE ARRAYS WITH DATA:
for url in urls:
  # Uncomment to see the "progress bar" :D
  #print(url)
  resp = requests.get(url)
  bs = BeautifulSoup(resp.text)

  date = bs.find('p', text = re.compile('DEALING DATE')).text
  dates.append(date.split()[2])

  nav = bs.find('p', text = re.compile('NAV PER SHARE')).text
  fnav = float(nav.split()[4])
  nav_per_share.append(fnav)

  shares = bs.find('p', text = re.compile('NUMBER OF SHARES')).text
  ishares = int(shares.split()[5])
  num_shares.append(ishares)

  total_navs.append(fnav*ishares)

In [12]:
# Test print:
for date in dates:
  print(date)

18-Feb-2020
17-Feb-2020
14-Feb-2020
13-Feb-2020
12-Feb-2020
11-Feb-2020
10-Feb-2020
07-Feb-2020
06-Feb-2020
05-Feb-2020
04-Feb-2020
03-Feb-2020
31-Jan-2020
30-Jan-2020
29-Jan-2020
28-Jan-2020
27-Jan-2020
24-Jan-2020
23-Jan-2020
22-Jan-2020
21-Jan-2020
20-Jan-2020
17-Jan-2020
16-Jan-2020
15-Jan-2020
14-Jan-2020
13-Jan-2020
10-Jan-2020
09-Jan-2020
08-Jan-2020
06-Jan-2020
03-Jan-2020
27-Dec-2019
23-Dec-2019
20-Dec-2019
19-Dec-2019
18-Dec-2019
17-Dec-2019
16-Dec-2019
12-Dec-2019
11-Dec-2019
10-Dec-2019
09-Dec-2019
06-Dec-2019
05-Dec-2019
04-Dec-2019
03-Dec-2019
02-Dec-2019
29-Nov-2019
28-Nov-2019
27-Nov-2019
26-Nov-2019
25-Nov-2019
22-Nov-2019
21-Nov-2019
20-Nov-2019
19-Nov-2019
18-Nov-2019
15-Nov-2019
14-Nov-2019
13-Nov-2019
12-Nov-2019
11-Nov-2019
08-Nov-2019
07-Nov-2019
06-Nov-2019
05-Nov-2019
31-Oct-2019
30-Oct-2019
29-Oct-2019
28-Oct-2019
25-Oct-2019
24-Oct-2019
23-Oct-2019
22-Oct-2019
21-Oct-2019
18-Oct-2019
17-Oct-2019
16-Oct-2019
15-Oct-2019
14-Oct-2019
11-Oct-2019
10-Oct-2019
09-O

In [0]:
#date = bs2.find('p', text = re.compile('DEALING DATE')).text
#date.split()[2]

In [0]:
#nav = bs2.find('p', text = re.compile('NAV PER SHARE')).text
#inav = float(nav.split()[4])
#inav

In [0]:
#shares = bs2.find('p', text = re.compile('NUMBER OF SHARES')).text
#n_shares = int(shares.split()[5])
#n_shares

In [0]:
#total_nav = inav * n_shares
#total_nav

In [0]:
#nrows = len(scripts)
#new_table = pd.DataFrame(columns=range(0,2), index = range(0, nrows))
#row_marker = 0
#for row in table.find_all('tr'):
#  column_marker = 0
#  columns = row.find_all('td')
#  for column in columns:
#    new_table.iat[row_marker,column_marker] = column.get_text()
#    column_marker += 1
#  row_marker+=1
#df = new_table.assign(date = pd.to_datetime(new_table[0], dayfirst=True))
#df.head(5)

In [0]:
#df = df.rename({1: "keyrate", 2: "date"}, axis='columns').drop(columns=[0]).dropna()
#df['keyrate'] = pd.to_numeric(df['keyrate'])
#df.index = pd.DatetimeIndex(df['date'])
#type(df.index)